In [1]:
import time
import datetime
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import numpy as np
import datetime as dt
from selenium.webdriver.common.by import By


def get_max_pages(driver):
    max_page = 1
    for i in driver.find_elements(By.CLASS_NAME, "ant-pagination-item"):
        if int(i.text) >= max_page:
            max_page = int(i.text)
    return max_page


def donki_scraper_with_category(driver, category):
    df = pd.DataFrame()
    driver.execute_script("window.scrollTo(0, 10);")
    time.sleep(3)
    for cate_list in driver.find_elements(By.CLASS_NAME, "type-item"):
        if cate_list.text == category:
            cate_list.click()
            time.sleep(3.5)

            category_max_page = get_max_pages(driver)
            print(cate_list.text,": ",category_max_page)
            time.sleep(3)
            for page in range(1,category_max_page+1):
                if page <= category_max_page:

                    driver.execute_script("window.scrollTo(0, 1060);")
                    time.sleep(1)
                    driver.execute_script("window.scrollTo(0, 1125);")
#                     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(3)

                    temp_df = pd.DataFrame()
                    category, goods_name, jpPrice, jporgPrice, referencePrice = [], [], [], [], []
                    for i in driver.find_elements(By.CLASS_NAME, "goods-item"): #product
                        goods_name.append(i.find_element(By.CLASS_NAME, "goodsName").text)
                        jpPrice.append(i.find_element(By.CLASS_NAME, "jpPrice").text)

                        try:
                            jporgPrice.append(i.find_element(By.CLASS_NAME, "orgPrice").text)
                        except NoSuchElementException:
                            jporgPrice.append(i.find_element(By.CLASS_NAME, "jpPrice").text)

                        referencePrice.append(i.find_element(By.CLASS_NAME, "referencePrice").text)
                        category.append(cate_list.text)

                    temp_df['category'] = category
                    temp_df['goods_name'] = goods_name
                    temp_df['jpPrice'] = jpPrice
                    temp_df['jporgPrice'] = jporgPrice
                    temp_df['referencePrice'] = referencePrice  
                    time.sleep(2)

                    df = pd.concat([df,temp_df],axis=0)
                    df = df.reset_index(drop = True)

                    if page < category_max_page:
                        driver.find_element(By.CLASS_NAME, "ant-pagination-next").click()
                        time.sleep(2)
                        driver.execute_script("window.scrollTo(0, 30);")
                        time.sleep(2)
                    else:
                        pass
                
            return df
        else: 
            pass


def pre_run(category):
    donki_link = 'https://mpglobal.donki.com/ec-web/d/pcd?titleStr=5YyW5aaG5ZOB&gpId=gm-0004?lan=zh-tw'
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(donki_link)
    time.sleep(3.5)
    driver.refresh()
    time.sleep(4)
    driver.get(donki_link)
    time.sleep(4)
    df = donki_scraper_with_category(driver = driver, category = category)
    df.to_csv('result/donki_scraper_result'+' category='+category+'.csv', index = False)

    driver.close()
    driver.quit()
    
    return df

In [3]:
df = pre_run(category = '美容護膚')



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/vincent/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache
/var/folders/xv/45cl7c1j79vdvpw52x0zzmv40000gn/T/ipykernel_73312/598458182.py:80: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


美容護膚 :  52


In [ ]:
df_necessities = pre_run(category = '日用品')
df_babymonther = pre_run(category = '母嬰用品')
df_health_care = pre_run(category = '衛生保健')
df_food = pre_run(category = '食品')
df_electronic = pre_run(category = '電器')
df_drug = pre_run(category = '藥品')
df_healthy = pre_run(category = '健康')
df_clothing = pre_run(category = '服飾')
df_toy = pre_run(category = '玩具')